# Party name extraction
---
Starting from the chunked text of a party's electoral manifesto, we aim to identify the official name of the party, assuming that it usually shows up at the start of the document, iterating from top to bottom.

## Setup

### Import libraries

In [ ]:
from loguru import logger
from tqdm.auto import tqdm
from pydantic import BaseModel, Field
from openai import OpenAI

In [ ]:
from polids.config import settings
from polids.party_name_extraction.openai import OpenAIPartyNameExtractor

## Load chunks to parse through

We're going to start from pre-chunked list of strings, so as to avoid dependencies on previous steps of the pipeline.

In [ ]:
chunks_by_party = {
    "livre": [
        "# Programa Eleitoral Legislativas 2022 LIVRE",
        "## Concretizar o Futuro\n### Uma sociedade justa num planeta saudável\nEste é o compromisso político do LIVRE para a próxima legislatura, com a defesa da justiça social e ambiental, com a igualdade e os direitos humanos, com a liberdade e com o futuro. A mudança de que precisamos para transformar Portugal, a Europa e o mundo começa com a mobilização em torno de um novo modelo de desenvolvimento para Portugal, assente na qualificação abrangente e avançada da população, em políticas que garantem justiça intergeracional, sustentabilidade ambiental e igualdade social. Este projeto de futuro tem uma visão ecologista, cosmopolita, libertária e universalista que antecipa os desafios do século XXI.",
        "A alternativa é LIVRE — não receamos compromissos políticos, e na Assembleia da República procuraremos, sempre, acordos amplos para concretizar as mudanças necessárias para um país mais justo e sustentável, e as políticas transformadoras de que necessitamos urgentemente.",
        "As eleições legislativas de 2022 acontecem num momento extremamente delicado para o país e num contexto exigente em que urge responder às devastadoras consequências da pandemia. Importa proteger os mais afetados pela crise económica, agravada pela crise emergente de energia e matérias-primas, acudir a um Serviço Nacional de Saúde em pré-rutura, enfrentar a crise ecológica e traçar uma estratégia de desenvolvimento inclusivo e sustentável no médio e longo prazos para Portugal. Esta exigência tem de ter uma resposta progressista à altura.",
        "A partir de 30 de janeiro a política progressista na Assembleia da República será LIVRE: com uma cultura de diálogo e de compromisso político em defesa do bem público e comum, que permitirá à esquerda lutar por conquistas concretas e palpáveis e ambicionar uma visão de futuro que nos dê esperança no mundo que construímos para as novas gerações.",
        "O LIVRE luta por uma mobilização ambiciosa, empática e humanista dos cidadãos para a transformação do nosso futuro colectivo. Queremos trabalhar a partir da Assembleia da República porque acreditamos que não há igualdade sem ecologia e liberdade sem democracia. Nestas legislativas a alternativa é LIVRE, o partido português da Esquerda Verde.",
        "## Índice\n- **Desenvolvimento Ecológico e Solidário**\n- **Igualdade, Justiça Social e liberdade**\n- **Trabalho, rendimento, Tempo e Proteção Social**\n- **Saúde**\n- **Educação**\n- **Conhecimento, Ciência e Ensino Superior**\n- **Cultura e Arte**\n- **Habitação e Espaço Público**\n- **Coesão Territorial, Transportes e Mobilidade**\n- **Emergência Climática e Energia**\n- **Economia Circular**\n- **Agricultura e Florestas**\n- **Conservação da Natureza e Biodiversidade**\n- **Bem-estar e Direitos dos Animais**\n- **Águas, rios e Oceanos**\n- **Justiça**\n- **Estado e instituições**\n- **Democracia**\n- **Prevenção e Combate à Corrupção**\n- **Soberania Digital**\n- **Portugal na Europa e no Mundo**",
        "## Desenvolvimento Ecológico e Solidário\nSó conseguimos combater a crise ecológica combatendo também a desigualdade social e alterando o modo como vivemos e como a sociedade está organizada. O crescimento económico de um país não assegura progresso ou desenvolvimento sustentável se não tiver em conta o bem-estar, a realização, a saúde e a felicidade da geração atual e das gerações futuras nem o uso sustentável dos recursos do planeta, a saúde dos ecossistemas e da biodiversidade, os ciclos naturais e o clima. Atualmente, o desempenho de um país é frequentemente apreciado com base apenas no que produz, através do seu Produto Interno Bruto (PIB), o que corresponde a uma visão redutora do desenvolvimento. Num planeta de recursos finitos, é necessário transitar para uma sociedade justa, que respeite os limites do planeta e que invista na realização pessoal, na qualidade de vida e no bem-estar de todos, de forma igual.",
    ],
    "volt": [
        "## Identidade e Visão do Volt\n\nImagina-te numa situação onde tu e os teus concidadãos teriam de desenhar de raiz a sociedade em que viveriam. Que tipo de sociedade escolherias?\n\nO filósofo John Rawls propôs que, antes de escolher, cada um de nós colocasse um ‘véu da ignorância’. Ao colocar este véu, desconheceríamos o nosso ponto de partida ou as nossas circunstâncias na sociedade, ou seja, não saberíamos qual o nosso nível de riqueza, o nosso estatuto, a nossa orientação sexual, não saberíamos nada sobre o nosso lugar no mundo. Não possuindo estas informações, e procurando o nosso próprio bem-estar, seria certamente consensual a construção de uma sociedade em que ter uma vida digna é um direito real e universal, onde a liberdade de qualquer pessoa é apenas limitada pelo direito à liberdade do nosso próximo e onde todos têm o mesmo nível de oportunidades para prosperar e alcançar a sua própria definição de felicidade.",
        "Esta é a **visão** do Volt de uma sociedade justa, de um Portugal e de uma Europa para todos, e esta funciona como ponto de partida para a ideologia que guia a nossa ação política.",
        "Somos **progressistas** porque não nos resignamos ao estado atual das coisas, opondo-nos ao conservadorismo e procurando promover soluções inovadoras para os nossos problemas comuns e para o aperfeiçoamento da condição humana em sociedade, soluções onde prevaleçam os valores da liberdade, da igualdade de oportunidades, da solidariedade, da dignidade humana e dos direitos humanos. Além de social, o progressismo que defendemos é um que inclui o **ecologismo** porque vemos o bem-estar do planeta e dos outros seres vivos como um fim em si mesmo e como uma condição necessária para o bem-estar da sociedade. Por isto mesmo, acreditamos que a necessidade de sustentabilidade impõe limitações importantes à forma como a sociedade é conduzida e em todas as soluções que defendemos temos a preocupação de encontrar uma harmonia com o que nos rodeia e com o que nos sustenta vendo como prioridade a preservação do meio ambiente, a utilização responsável de recursos minerais e energéticos e o combate às alterações climáticas.",
        "Somos **pragmáticos** porque procuramos ser coerentes com a nossa visão e valores propondo e defendendo o que de facto funcione com base no consenso, no uso da razão e da evidência científica e nunca em dogmatismos ideológicos ou no egoísmo na procura do poder. Colocamo-nos ao centro do espetro político na forma como vemos o papel do Estado, pois vemos este como um meio e não como um fim, acreditamos que o Estado deve intervir tão pouco e tão rápido quanto possível e tanto e por quanto tempo for necessário.",
        "Somos **europeístas** porque, sendo Portugueses, sentimo-nos também Europeus. Vemos a União Europeia como parte da nossa identidade e futuro e, reconhecendo a interdependência do mundo em que vivemos, olhamos para o aprofundamento do projeto europeu e para a construção de uma federação europeia como o caminho para a resolução dos grandes desafios que enfrentamos. Queremos uma Europa unida e mais democrática que valoriza os seus cidadãos e os seus residentes, que não deixa ninguém para trás e que cria e mantém as condições necessárias para a realização do potencial único de cada um e da comunidade no seu todo. Uma Europa que se esforça continuamente para alcançar, em conjunto e de forma solidária, os mais elevados padrões de desenvolvimento humano, social, ambiental e técnico e que possa ajudar a guiar o resto do mundo através do exemplo na prossecução desses mesmos padrões, sendo um promotor chave da paz, da justiça, da prosperidade e da sustentabilidade global.",
        "Em 2017, como resposta aos movimentos nacionalistas e populistas, e em específico como resposta ao Brexit, um conjunto de jovens de várias nacionalidades criou este movimento político a que foi dado o nome de Volt. Foi escolhido este nome porque este simboliza, da mesma forma por toda a Europa, aquilo que queremos fazer, dar pelas nossas próprias mãos uma nova energia à política.",
    ],
}

## Get the party name

### Initial test

In [ ]:
client = OpenAI(api_key=settings.openai_api_key)

In [ ]:
class PartyName(BaseModel):
    """
    Model to represent the name of a political party, in both full and short forms.
    """

    full_name: str = Field(description="Full name of the party")
    short_name: str = Field(
        description="Short name of the party, abbreviated in an acronym"
    )
    is_confident: bool = Field(
        description="Whether the model is confident in its answer or not. e.g. if the party name is not in the text or it's ambiguous, then this should be False.",
    )

In [ ]:
party_names: list[PartyName] = []
batch_size = 2
for original_party_name, chunks in tqdm(
    chunks_by_party.items(), desc="Processing party chunks"
):
    is_confident = False
    idx = 0
    previous_guess: PartyName = PartyName(
        full_name="", short_name="", is_confident=False
    )
    while not is_confident:
        current_chunks = chunks[idx : idx + batch_size]
        completion = client.beta.chat.completions.parse(
            # Using the mini version for cheaper processing; setting a specific version for reproducibility
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant specialized in analyzing political manifestos to identify the name of the political party that authored the document. Your task is to extract the party's name from the provided <manifesto_text>, ensuring that the identification is confident and unambiguous. If the party name cannot be confidently identified, or if the text is vague, ambiguous, or mentions a party not affiliated with the manifesto, you must label the result as not confident (`is_confident = False`). Use the <previous_guess> to refer to the previous, non-confident guess of the party name as memory to refine your attempts.",
                },
                {
                    "role": "user",
                    "content": f"""**Overall Goal:**
Analyze the <manifesto_text> (which is in Markdown format from a political manifesto) and identify the name of the political party that authored it. **The identification must be confident and unambiguous.** If the text is vague, ambiguous, or mentions a party not affiliated with the manifesto, label the result as not confident (`is_confident = False`). Use the <previous_guess> to refer to the previous, non-confident guess of the party name as memory to refine your identification attempts.

**Analysis Process:**
1. **Carefully Analyze:** Read and fully understand the <manifesto_text> to grasp the context and identify any explicit mentions of the party's name.
2. **Check for Confidence:** Determine if the party's name is presented in a clear and unambiguous way. If the identification is not confident, label the result as `is_confident = False`.
3. **Use Previous Guesses:** If a previous, non-confident guess of the party name exists, use the <previous_guess> as memory to refine your identification attempts.
4. **Avoid False Positives:** Ensure that the identified name belongs to the party that authored the manifesto and not to another party mentioned in the text.
5. **Format Output:** Construct the required structured output according to the provided schema.

**Critical Constraints:**
- **Exact Markdown Preservation:** You MUST NOT alter the <manifesto_text>. Write the party name as it appears in the text, without any modifications.
- **Language Agnostic:** Preserve the original language as in the <manifesto_text> within the output.

<manifesto_text>
{current_chunks}
</manifesto_text>

<previous_guess>
    <full_name>{previous_guess.full_name}</full_name>
    <short_name>{previous_guess.short_name}</short_name>
    <is_confident>{previous_guess.is_confident}</is_confident>
</previous_guess>""",
                },
            ],
            response_format=PartyName,  # Specify the schema for the structured output
            temperature=0,  # Low temperature should lead to less hallucination
            seed=42,  # Fix the seed for reproducibility
        )
        party_name_guess = completion.choices[0].message.parsed
        assert isinstance(party_name_guess, PartyName), (
            "Output does not match the expected schema."
        )
        is_confident = party_name_guess.is_confident
        if is_confident:
            # If the model is confident, append the result to the list and move on to the next political party
            party_names.append(party_name_guess)
            if (
                party_name_guess.short_name != original_party_name
                and party_name_guess.full_name != original_party_name
            ):
                logger.warning(
                    f"The model guessed a different party name instead of {original_party_name}:\n{party_name_guess.model_dump()}"
                )
        else:
            # If the model is not confident, use it as the previous guess
            logger.info(
                f"The model was not confident in its #{idx} guess:\n{party_name_guess.model_dump()}"
            )
            previous_guess = party_name_guess
            idx += batch_size
party_names

### Implemented solution

In [ ]:
party_name_extractor = OpenAIPartyNameExtractor()
party_names = {
    original_party_name: party_name_extractor.extract_party_names(chunks)
    for original_party_name, chunks in chunks_by_party.items()
}
party_names